## Analysis with GeoPandas
In this exercise, we use GeoPandas to identify census tracts within 1km of EV charging locations in a region in NC. 
* [Input/Output](https://geopandas.org/io.html) Reading a CSV file into a GeoPandas geodataframe
* [Index and Selecting data](https://geopandas.org/indexing.html): Subsetting records
* [Managing Projections](https://geopandas.org/projections.html): Projecting a geodataframe
* [Geometric manipulations](https://geopandas.org/geometric_manipulations.html): Buffering points
* [Set operations](https://geopandas.org/set_operations.html): intersect
* [Aggregrating data](https://geopandas.org/aggregation_with_dissolve.html): Dissolving features

## Part 1: Fetching and Exploring the Data
Here we'll gather and explore the data we'll be using in our analysis. This includes two datasets. First is the list of EV Charging locations, stored as a CSV file in our data folder. This dataset has coordinate columns that we'll use to construct points and convert into a geodataframe.

The second dataset is comprised of 2010 Census BlockGroup data for all of North Carolina. We'll fetch these data from an on line resource using a web service. We'll revisit how web services later; for now, we'll use this process to fetch data for three counties: Durham, Wake, and Orange. 

For each dataset, we'll get the data into geodataframe format and then explore the data in various ways. Then we'll move to Part 2 where we analyse the data. 

### ♦Step 1. Import packages needed in the analysis
>**Knowledge check**:<br>
→ Can you explain what role each package imported might do in our analysis?

In [ ]:
#Import packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
import contextily as ctx

%matplotlib inline

### ♦Step 2. Create a geodataframe from a CSV file
As done in a previous notebook, we want to:
* Import a CSV file containing coordinate columns into a Pandas dataframe,
* Create a collection of Shapely points from the coordinate fields, and 
* Create a geodataframe from the components. 

In [ ]:
#Read in charging stations CSV, convert to geodataframe
df = pd.read_csv('./data/NC_Charging_Stations.csv')
geom = [Point(xy) for xy in zip(df['Longitude'],df['Latitude'])]
gdf_stations_all = gpd.GeoDataFrame(df,geometry=geom,crs=4326)

### ♦Step 3. Explore the data in geodataframe
Have a quick look at the contents imported. Things to check include:
* How many rows and columns were imported
* The names, data types, and number of non-null values in each column
* Summary statistics of numeric columns, if applicable
* Correlations among column values, if applicable
* Spatial plot of the data

In [ ]:
#Examine the data
gdf_stations_all.info()

In [ ]:
#Plot the data
gdf_stations_all.plot();

>### ►TASK: Import the USGS gage points for NC: `./data/gages.csv`
> * Convert data to a geodataframe: `gdf_gages`
> * Explore the data
> * Plot the gage sites

---
### ♦Step 4. Import NC Census Block Group features via NC OneMap's web service
_We will explore web services a bit later, but we'll use the code below to acquire polygon data of census block groups for Durham, Wake, and Orange counties from an NC OneMap Web Service. Once imported, we'll merge these geodataframes together and use them in our subsequet analyses._

* First, to simplify matters, I've created a Python function to fetch data for a specific county given its FIPS code.

In [ ]:
#Create a function to read NCOneMap feature services into a geodataframe
def getBlockGroupData(FIPS):
    #Construct the url from the function arguments
    url=f'https://services.nconemap.gov/secure/rest/services/NC1Map_Census/FeatureServer/8/query?' + \
        f"where=GEOID10+LIKE+'{FIPS}%'&outFields=GEOID10,TOTAL_POP&f=geojson"
    
    #Create a geodataframe from the URL
    gdf = gpd.read_file(url)
    
    #Return the geodataframe
    return gdf

* Now, we apply that function for the three counties we want to examine

In [ ]:
#Fetch census block groups for Durham, Orange, and Wake counties using the above function
gdf_DurmBlkGroups = getBlockGroupData(37063)
gdf_WakeBlkGroups = getBlockGroupData(37183)
gdf_OrangeBlkGroups = getBlockGroupData(37135)

* _Challenge: See if you can fetch Chatham county block groups (FIPS = 37037)_

In [ ]:
#Challenge: See if you can fetch Chatham county block groups (FIPS = 37037)


* **Explore** the data...
 * What is its coordinate reference system?
 * What columns are included?
 * What does the first record look like?

In [ ]:
#Show the Durham block group geodataframe's coordinate reference system
gdf_DurmBlkGroups.crs

In [ ]:
#Explore the Durham block group geodataframe's columns...
gdf_DurmBlkGroups.info()

In [ ]:
#Examine a sample record from the geodataframe
gdf_DurmBlkGroups.iloc[0]

* **Visualize** the data...

In [ ]:
#Plot Durhham's population
gdf_DurmBlkGroups.plot('TOTAL_POP',cmap='viridis')

In [ ]:
#Plot the block groups for all three counties
thePlot = gdf_DurmBlkGroups.plot(color='blue')
gdf_WakeBlkGroups.plot(ax=thePlot,color='red')
gdf_OrangeBlkGroups.plot(ax=thePlot,color='lightblue');

## Part 2: The analysis
Now that we've obtained a few datasets and got them into geodataframes, we'll perform some analysis. These include:
* Subsetting the EV charging stations for those in specific cities.
* Identifying the census blocks surrounding each EV station, within a distance of 1km
 * To do this, we'll merge the Durham, Wake, and Orange Co block data selected above
 * Then we'll buffer our selected EV station points a distance of 1km
 * And finally, we'll select blocks that intersect the EV station buffers

### Analysis 1: Subset the EV Station points based on attribute values
Doc: https://geopandas.org/indexing.html

Subsetting features in a geodataframe uses the same methods as subsetting recordsin a Pandas dataframe. Here we'll run through an example by subsetting EV stations found oly within Durham, Raleigh, and Chapel Hill. 

* **Step 1** Examine unique values in the `City` column

In [ ]:
#Reveal the unique values in the City column
gdf_stations_all['City'].unique()

* **Step 2** Subset records for those where the City is "Durham", "Raleigh", or "Chapel Hill"

In [ ]:
#Subset records where the City is "Durham", "Raleigh", or "Chapel Hill"
gdf_stations = gdf_stations_all.query('City in ("Durham","Raleigh","Chapel Hill")')

* **Step 3** Explore the results...

In [ ]:
#Plot the results
gdf_stations.plot("City");

In [ ]:
#Plot them with a base map
fig, ax = plt.subplots(figsize = (10,5))
gdf_stations.to_crs(3857).plot(ax=ax, column="City")
ctx.add_basemap(ax)

### Analysis 2. Merge the 3 county block group geodataframes into one
Doc: https://geopandas.org/mergingdata.html
1. Check that all data have the same crs
1. Optionally, add a field to identify the source geodataframe
1. Apply the `append()` function
1. Check/explore the result

We'll start by appending the Wake Co. dataset to the Durham Co. one. Then you will append the Orange Co. dataframe to that product.

* **Step 1** Check that the two files share the same coordinate reference system

In [ ]:
#Check the crs of the two geodataframes
gdf_DurmBlkGroups.crs == gdf_WakeBlkGroups.crs

* **Step 2** Add an identifying column to the source geodataframes

In [ ]:
#Add a field to each input, setting values to identify the source dataset
gdf_DurmBlkGroups['County'] = 'Durham'
gdf_WakeBlkGroups['County'] = 'Wake'

* **Step 3** Append one dataframe to the other

In [ ]:
#Append the Wake Co features to the Durham Co features,
gdf_BlkGrp_step1 = gdf_DurmBlkGroups.append(gdf_WakeBlkGroups)

* **Step 4** Explore the result

In [ ]:
#Check to see that the total rows in the merged gdf match the sum of the two component gdfs


In [ ]:
#Plot the result
gdf_BlkGrp_step1.plot('County');

##### TASK:
_Now you try to append the Orange Co blockgroup features to the `gdf_BlkGrp_step` geodataframe we just created._

**Remember to:**
* check that the coordinate refernce systems are the same, and 
* add a new column to the `gdf_OrangeBlkGroups`, setting its value to the County name.
 
→ Save the result as `gdf_BlkGrps`

In [ ]:
#Check that the coordinate refernce systems are the same
gdf_BlkGrp_step1.crs == gdf_OrangeBlkGroups.crs

In [ ]:
#Add the county field
gdf_OrangeBlkGroups['County'] = 'Orange'

In [ ]:
#Append the geodataframes
gdf_BlkGrp = gdf_BlkGrp_step1.append(gdf_OrangeBlkGroups)

In [ ]:
#Plot the output
gdf_BlkGrp.plot('County');

### Analysis 3: Dissolve block features to the tract level
We have Social Vulnerability Data to examine in our analysis, but these data are at the Tract, not BlockGroup level. Thus, to join these attributes to our geodataframe, we'll need to aggregate our blockgroups to the tract level. Fortunately, the `GEOID10` attribute is structured such that the census tract is just the first 11 characters. So we will create a new column holding these first 11 characters, and then we'll dissolve our blockgroup features sharing the same tract ID to single features.

Doc: https://geopandas.org/aggregation_with_dissolve.html
* First, create a new column listing tract IDs (the first 11 digits of the GEOID10)
* Dissolve the features on this attribute, computing aggregate sum of the TOTAL_POP field

* **Step 1** Create the Tract column from the GEOID10 values

In [ ]:
#Create the Tract column
gdf_BlkGrp['TRACT']=gdf_BlkGrp['GEOID10'].str[:11]
gdf_BlkGrp.head()

* **Step 2** Dissolve features on the Tract column

In [ ]:
#Dissolve features on tract, computing summed population
gdf_Tract = gdf_BlkGrp.dissolve('TRACT',aggfunc={'TOTAL_POP':'sum','County':'first'})
gdf_Tract.head()

In [ ]:
#Plot the data
gdf_Tract.plot('TOTAL_POP',cmap='YlGnBu')

### Analysis 4: Import the Social Vulnerability Data and join with the Tract features
Now that we have the data at the tract level, we can join the Social Vulnerability Index data, stored in a CSV file (`./data/NC_SVI_2018.csv`).

Doc: https://geopandas.org/mergingdata.html#attribute-joins
* Import the SVI data as a Pandas dataframe
* Append records from the SVI dataframe to the Tracts geodataframe

* **Step 1** Import and explore the SVI data into a Pandas dataframe

In [ ]:
#Import and explore the SVI data
df_SVI = pd.read_csv('./data/NC_SVI_2018.csv',dtype={'FIPS':'str',})
df_SVI.info()

>**Challenge**:<br>→ _Modify the `read_csv()` command above so that 'ST' and 'STCNTY' are also imported as strings._

In [ ]:
#Plot a histogram of the SVI values
df_SVI['SVI'].hist();

<h3><font color='red'> Whoops!!! </h3></font>
Values should be between 0 and 1, but we see in the histogram that a few value are down near -1000. Turns out a few records have SVI values of -999. We need to remove those records.

In [ ]:
#Create a mask of values greater than or equal to zero
valid_mask = df_SVI['SVI'] >= 0
#Apply that mask
df_SVI_fixed = df_SVI.loc[valid_mask]

In [ ]:
#View the histogram again
df_SVI_fixed['SVI'].hist();

_Phew! Exploring the data payed off!_

* **Step 2** Append the dataframe to the tracts

In [ ]:
#Have a look at the merge command syntax
gdf_Tract.merge?

In [ ]:
gdf_Tract.shape

In [ ]:
#Join the SVI data to the tract features
gdf_Tract_joined = gdf_Tract.merge(df_SVI_fixed,
                                   left_on='TRACT', #"Join to" field in the traft features 
                                   right_on='FIPS', #"Join on" field in the SVI dataframe
                                   how='left')      #Keep all tract features, even if they are missing SVI
#Examine the output
gdf_Tract_joined.head()

In [ ]:
#Explore the output
gdf_Tract_joined.info()

In [ ]:
#Plot the output
gdf_Tract_joined.plot('SVI',figsize=(7,7));

_Looks like we have some features missing SVI data. Let's examine those more closely._

In [ ]:
#Create a mask of null SVI values
gdf_Tract_joined['SVI'].isnull()

In [ ]:
#Apply the mask
gdf_Tract_joined.loc[gdf_Tract_joined['SVI'].isnull()]

_We can either assign a value to these missing values or leave them as no data. We'll just leave them blank for now..._

### Analysis Step 5: Compute Population Density for Each Tract
Our combined dataframes have a field indicating the total population in each block group. We want to compute population density from this and from the area of each tract. We don't yet have an area field in our dataframe, but we can compute that from our spatial features. But before we can do this, we need to transform our data into a projected coordinate system. So... the steps for this analysis include:
* Project the dataframe from WGS84 to UTM Zone 17N
* Compute a new `Area_km2` column in our dataframe
* Compute a new `PopDens` column in our dataframe by dividing `TOTAL_POP` by `Area_km` 

In [ ]:
#Project the data to UTM Zone 17N (EPSG 32617)
gdf_Tract_utm = gdf_Tract_joined.to_crs(32617)

In [ ]:
#Compute a new column of geometry area (in sq km)
gdf_Tract_utm['Area_km2'] = gdf_Tract_utm['geometry'].area / 1000000

In [ ]:
#Compute a new column of population density
gdf_Tract_utm['PopDens'] = gdf_Tract_utm['TOTAL_POP'] / gdf_Tract_utm['Area_km2']

In [ ]:
#Plot the distribution of areas
gdf_Tract_utm['PopDens'].hist(bins=20);

In [ ]:
#Plot a map of log tranformed population density
gdf_Tract_utm.plot('PopDens',figsize=(10,8),cmap='viridis');

In [ ]:
#Log transform the pop_dens data
import numpy as np
gdf_Tract_utm['PD_log'] = np.log(gdf_Tract_utm['PopDens'])

In [ ]:
#Plot the log-transformed distribution of areas
gdf_Tract_utm['PD_log'].hist(bins=20);

In [ ]:
#Plot a map of log tranformed population density
gdf_Tract_utm.plot('PD_log',figsize=(10,8),cmap='viridis');

In [ ]:
gdf_Tract_utm.columns

#### Analysis: Subset EV stations spatially. 
Doc: https://geopandas.org/set_operations.html
Previously, we subset EV stations by an attribute (City). Here we'll see how we can instead select features spatially. We do this with GeoPanda's Overlay operations.

To spatially select features:
* Ensure both datasets share the same coordinate reference system; transform if needed
* 

In [ ]:
#plot points on tracts
ax = gdf_Tract_utm.plot(figsize=(12,6),alpha=0.3)
gdf_stations_utm.plot(column='City',ax=ax,markersize=12);

* Get both datasets into the same crs

In [ ]:
#Ensure both datasets share the same crs
print(gdf_stations_all.crs, gdf_Tract_utm.crs)

In [ ]:
#Project one dataset to match the other
gdf_stations_all_utm = gdf_stations_all.to_crs(gdf_Tract_utm.crs)
print(gdf_stations_all_utm.crs)

* Select EV stations that intersect the county features.

In [ ]:
#Show info on the overlay function
gdf_stations_select = gpd.overlay?

In [ ]:
#Intersect the two dataframes
gdf_stations_select = gpd.overlay(
    df1=gdf_stations_all_utm,
    df2=gdf_Tract_utm,
    how='intersection'
)

In [ ]:
gdf_stations_select.head()

In [ ]:
#Plot
ax = gdf_Tract_utm.plot(color='lightgrey',edgecolor='grey',alpha=0.4,figsize=(12,12))
gdf_stations_select.plot(ax=ax,column='City',markersize=45,edgecolor='white');
ctx.add_basemap(ax, crs=gdf_Tract_utm.crs,source=ctx.providers.CartoDB.Voyager)

#### Analysis: Spatial Join
Doc: https://geopandas.org/mergingdata.html#spatial-joins

Now that we have a proper susbset of EV stations, let's examine the block groups in which each EV stations falls. We do this with a Spatial Join 

In [ ]:
#Get syntax of sjoin function
gpd.sjoin?

In [ ]:
#Buffer the selected sites
gdf_stations_select['geometry'] = gdf_stations_select.buffer(1000)

In [ ]:
#Execute the spatial join
gdf_JoinedData = gpd.sjoin(
    left_df = gdf_stations_select,
    right_df = gdf_Tract_utm,
    how='inner',
    op='intersects',
    lsuffix='ev',
    rsuffix='census'
)

In [ ]:
gdf_JoinedData.head()

In [ ]:
gdf_JoinedData[['SVI_right','PopDens_right']].plot(kind='scatter',x='SVI_right',y='PopDens_right',color='City')

In [ ]:
#Plot
ax = gdf_Tract_utm.plot(color='lightgrey',edgecolor='grey',alpha=0.4,figsize=(12,12))
gdf_JoinedData.plot(ax=ax,column='SVI_right',markersize='SVI_right',edgecolor='white');
ctx.add_basemap(ax, crs=gdf_Tract_utm.crs,source=ctx.providers.CartoDB.Voyager)